# Glidelytics for Strava - paragliding analysis on personal Strava data
## Authorization 

Get client_id from
https://www.strava.com/settings/api

Visit page 
http://www.strava.com/oauth/authorize?client_id=[REPLACE_WITH_YOUR_CLIENT_ID]&response_type=code&redirect_uri=http://localhost/exchange_token&approval_prompt=force&scope=read_all
replacing placeholder with client_id

Get field "code"after redirection

Open file config.ini (or else create such file on root using example_config.ini). Update the file with the below fields and save:
- client_id
- client_secret
- code (from authorization response)
- expires_at (to check if needed to refresh)

In [50]:
import datetime
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import pandas as pd
from pandas import json_normalize

auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"

In [47]:
# Lets load the config info and token expiracy time
import configparser
config = configparser.ConfigParser()
config.read("config.ini")
print("Client ID: {}".format(config["strava"]["client_id"]))

expires_at_epoch = int(config["strava"]["expires_at"])
expires_at_date = datetime.datetime.fromtimestamp(expires_at_epoch)
refresh_token = config["strava"]["refresh_token"]
access_token = config["strava"]["access_token"]
print("Token expires at: {}".format(expires_date))

Client ID: 80263
Token expires at: 2022-04-04 20:42:57


In [48]:
# If a token is outdated we need to refresh it (happens often at the start of a script)
if datetime.datetime.now() >= expires_at_date:
    print("Refreshing token")
    payload = {
        'client_id': config["strava"]["client_id"],
        'client_secret': config["strava"]["client_secret"],
        'refresh_token': refresh_token,
        'grant_type': "refresh_token",
        'f': 'json'
    }

    try:
        response = requests.post(auth_url, data=payload, verify=False).json()
        access_token = response['access_token']
        refresh_token = response['refresh_token']
        expires_at_epoch = int(response['expires_at'])
        expires_at_date = datetime.datetime.fromtimestamp(expires_at_epoch)
        print("Token refreshed successtully and expires at: {}".format(expires_at_date))
    except:
        print("Problem refreshing token")

Refreshing token
Token refreshed successtully and expires at: 2022-04-05 03:39:33


In [49]:
print(access_token)

#request header for actitivies list
header = {'Authorization': 'Bearer ' + access_token}


6c43aa56101f85934014b2b1a9fa9940ea4c72e7


## Getting list of paragliding activities

In [56]:
# estimate of number of activities
act_total = 400
# activities per page
act_pp = 100
# number of pages
import math
pgs = math.ceil(act_total / act_pp)

# columns to keep from activities
cols = ["id", "name", "distance", "moving_time", "elapsed_time", "total_elevation_gain", "type", "start_date_local", "map.id", "average_speed", "max_speed", "elev_high", "elev_low"]

pg_contents = []
for pg in range(pgs):
    if datetime.datetime.now() >= expires_at_date:
        print("Token outdated")
        break
    print("Page: {}".format(pg + 1))
    param = {'per_page': act_pp, 'page': pg + 1}
    pg_dataset = requests.get(activites_url, headers=header, params=param).json()
    #print(pg_dataset)
    pg_activities = json_normalize(pg_dataset)
    pg_activities = pg_activities[cols]
    print("retrieved {} activities".format(pg_activities.shape[0]))
    print(pg_activities.head())
    pg_contents.append(pg_activities)
    
my_activities = pd.concat(pg_contents)
print("Total of {} activities retrieved".format(my_activities.shape[0]))

Page: 1
   resource_state                name  distance  moving_time  elapsed_time  \
0               2      Afternoon Hike       0.0           88            88   
1               2        Morning Hike    4980.0         4261          7040   
2               2  Afternoon Activity    6536.0         6050         15264   
3               2           Lunch Run    9760.0         3420          3420   
4               2           Lunch run    9766.6         3461          3482   

   total_elevation_gain     type          id            start_date  \
0                   0.0     Hike  6906273468  2022-03-30T12:16:20Z   
1                   0.0     Hike  6895088466  2022-03-27T07:13:53Z   
2                 707.0  Workout  6886935853  2022-03-26T12:55:13Z   
3                   0.0      Run  6878705037  2022-03-24T11:47:21Z   
4                  32.3      Run  6869475041  2022-03-23T11:49:38Z   

       start_date_local  ... has_kudoed  athlete.id athlete.resource_state  \
0  2022-03-30T14:16:20Z 

In [57]:
# Store data in file
my_activities.to_csv("My_activities.csv")

In [58]:
# get description for last 800 activities (where my paragling ones are)
my_parag_activities = my_activities.tail(90)
for index, row in my_parag_activities.iterrows():
    

            id                name  distance  moving_time  elapsed_time  \
70  3467638436           Lunch Run    2946.0         3673          4164   
71  3460617106           Lunch Run    9835.0         8892         12590   
72  3449780278       Afternoon Run    5115.0         2985          3197   
73  3418638352          Lunch Ride   19698.0         3752          4601   
74  3418772803        Morning Ride   13592.0         2055          2202   
..         ...                 ...       ...          ...           ...   
55  1867329459        Evening Ride    5101.0         1278          1774   
56  1866893147  Afternoon Activity     238.0          718           718   
57  1789020090      Giro mattutino    5470.3         1098          1098   
58  1764912272   464 atras de casa   18142.3        12577         19451   
59  1758839279   ride_201808081915    8659.0         1606          1653   

    total_elevation_gain     type      start_date_local       map.id  \
70                 468.0   